In [5]:
import requests
import re
from random import randint
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

In [6]:
#def data_film(genre='',sort='', rating='', subscript='available_online', country='', kind=''(film/serial), page=''):
def data_film(l):
    l = l.split('| ')
    url = f"https://www.kinopoisk.ru/lists/navigator/{l[0]}/?sort={l[1]}&quick_filters=%2C{l[2]}%2C{l[3]}%2C{l[4]}%2C{l[5]}&tab=online&page={l[6]}"
    dr = webdriver.Chrome()
    try:
        dr.get(url)
    except:
        return ([])
   # print(url)
    soup = BeautifulSoup(dr.page_source)
    res = soup.find_all('div', {'class' :'desktop-seo-selection-film-item__upper-wrapper'})
    return (res)

In [7]:
def get_row(i, raw_data):
    name = raw_data[i].find_all('p', {'class' : 'selection-film-item-meta__name'}, limit = 1)
    if (len(name) > 0): name = name[0].text 
    else: return 0
    
    link = raw_data[i].find_all('a', {'class' : 'selection-film-item-meta__link'}, limit = 1)
    if (len(link) > 0): link = 'https://www.kinopoisk.ru' + link[0].get('href') 
    else: return 0
    
    date = raw_data[i].find_all('p', {'class' : 'selection-film-item-meta__original-name'}, limit = 1)
    if (len(date) > 0): date = date[0].text 
    else: return 0
    
    date = re.search(r'[\d]{4}$', date)
    if (date != None): date = date.group(0) 
    else: return 0
    
    x = raw_data[i].find_all('span', {'class' : 'selection-film-item-meta__meta-additional-item'}, limit = 2)
    if (len(x) > 1): country = x[0].text; gen = x[1].text 
    else: return 0
    
    rating = raw_data[i].find_all('span', {'class' : r'rating__value rating__value_positive'}, limit = 1)
    if (len(rating) == 0): rating = raw_data[i].find_all('span', {'class' : r'rating__value rating__value_neutral'}, limit = 1)
    if (len(rating) > 0): rating = rating[0].text 
    else: return 0
    
    rating_count = raw_data[i].find_all('span', {'class' : 'rating__count'}, limit = 1)
    if (len(rating_count) > 0): rating_count = rating_count[0].text 
    else: return 0
    
    return [name, date, country, gen, rating, rating_count, link]

In [8]:
def get_add_info(df, name):
    if name not in list(df1.name):
        return('Something wrong')
    index = df.loc[(df.name == name)].index[0]
    if str(df.iloc[index]['audio']) != '0':
        return df
    link = df.iloc[index]['link']
    dr = webdriver.Chrome()
    try:
        dr.get(link)
    except:
        return('Something wrong')
    soup1 = BeautifulSoup(dr.page_source)
    info = soup1.find_all('div', {'class' : 'styles_valueLight__1j0RO'}, limit = 2)
    if (len(info) > 1): audio_track = info[0].text; subtitles = info[1].text
    else: return('Something wrong')
    description = soup1.find_all('p', {'class' : 'styles_paragraph__2Otvx'}, limit = 1)
    if (len(description) > 0): description = description[0].text
    else: return('Something wrong')
    
    df.iloc[index]['audio'] = audio_track
    df.iloc[index]['subtitles'] = subtitles
    df.iloc[index]['description'] = description
    return df


def add_info_to_str(df, name):
    index = df.loc[(df.name == name)].index[0]
    elem = df.iloc[index]
    country = elem['country']
    date = elem['date']
    audio_track = elem['audio']
    subtitles = elem['subtitles']
    description = elem['description']
    s = name + '\\n\\n'
    s = 'Страна:\\n' + country + '\\n\\n'
    s += 'Год:\\n' + date + '\\n\\n'
    s += 'Язык озвучки:\\n' + audio_track + '\\n\\n'
    s += 'Субтитры:\\n' + subtitles + '\\n\\n'
    s += 'Описание:\\n' + description
    return s

In [111]:
s = '| | | yandex_plus_subscription| foreign| | '.split('| ')
print('| '.join(s))
# '| | | yandex_plus_subscription| foreign| | serials'

| | | yandex_plus_subscription| foreign| | 


In [115]:
s = get_add_info(df, 'Ford против Ferrari')
print(s)

Страна:\nСША\n\nГод:\n2019\n\nЯзык озвучки:\nРусский, Английский\n\nСубтитры:\nРусские, Английские\n\nОписание:\nВ начале 1960-х Генри Форд II принимает решение улучшить имидж компании и сменить курс на производство более модных автомобилей. После неудавшейся попытки купить практически банкрота Ferrari американцы решают бросить вызов итальянским конкурентам на трассе и выиграть престижную гонку 24 часа Ле-Мана. Чтобы создать подходящую машину, компания нанимает автоконструктора Кэррола Шэлби, а тот отказывается работать без выдающегося, но, как считается, трудного в общении гонщика Кена Майлза. Вместе они принимаются за разработку впоследствии знаменитого спорткара Ford GT40.


In [9]:
def get_ten(num, l):
    l += '| ' + str((num - 1) // 50 + 1)
    num = (num - 1) % 50 ;
    raw_data = data_film(l)
    if (len(raw_data) == 0):
        return (0)
    df = pd.DataFrame(columns=['name','date','country','genre','rating',\
'rating_count', 'link', 'audio', 'subtitles', 'description', 'users'])
    max_ind = len(raw_data)
    count = 0
    while (num < max_ind and count < 10):
        a = get_row(num , raw_data)
        if (a):
            a.extend([0, 0, 0, 1])
            df.loc[count] = a
            count += 1
        num += 1
    return df

In [62]:
#data_film(genre='',sort='', rating='', subscript='available_online', country='', kind=''(film/serial), page=''):
genre = ['', 'anime', 'mystery', 'comedy', 'crime', 'adventure', 'family', 'thriller', 'horror', 'sci-fi', 'fantasy']
sort = ['', 'popularity', 'year']
rating = ['', 'high_rated']
subscript= ['available_online', 'yandex_plus_subscription', 'kinopoisk_with_amediateka_subscription']
country = ['', 'foreign', 'russian']
kind = ['', 'films', 'serials']

In [10]:
%%time
search_param = '| | | yandex_plus_subscription| foreign| | '
#data_film(genre='',sort='', rating='', subscript='available_online', country='', kind=''(film/serial), page=''):
d1 = get_ten(51, search_param)
d2 = get_ten(58, search_param)

CPU times: user 782 ms, sys: 48.9 ms, total: 831 ms
Wall time: 21.1 s


In [77]:
s = '|222|111|333|444|555|666|777|888|999|000'
s2 = '222'
if s2 not in s:
    s += '|' + s2
print(s)
s = re.sub('\|222', '', s)
print(s)

|222|111|333|444|555|666|777|888|999|000
|111|333|444|555|666|777|888|999|000


In [4]:
s = 'null'
s = s.split('|')
s

['null']

In [50]:
%%time
print('Ford против Ferrari' in list(df1['name']))
# df = pd.concat([df1, df2]).drop_duplicates('name').reset_index(drop=True)
df1['audio'] = df1['audio'] - df1['name'].isin(df2['name'])
df1

False
CPU times: user 4.24 ms, sys: 586 µs, total: 4.83 ms
Wall time: 4.54 ms


,name,date,country,genre,rating,rating_count,link,audio,subtitles,description
1,Легенда,2015,"Великобритания, Франция","криминал, триллер",7.2,387 977,https://www.kinopoisk.ru/film/839954/,0,0,0
2,Алита: Боевой ангел,2019,"США, Япония","фантастика, боевик",7.1,384 969,https://www.kinopoisk.ru/film/88173/,0,0,0
3,Довод,2020,"Великобритания, США","фантастика, боевик",7.6,384 587,https://www.kinopoisk.ru/film/1236063/,0,0,0
4,Пираты Карибского моря: На краю света,2007,США,"приключения, боевик",8.0,379 709,https://www.kinopoisk.ru/film/102124/,-1,0,0
5,Дэдпул 2,2018,"США, Канада","фантастика, боевик",7.4,376 478,https://www.kinopoisk.ru/film/961715/,-1,0,0
6,Богемская рапсодия,2018,"Великобритания, США","биография, музыка",8.0,375 027,https://www.kinopoisk.ru/film/568289/,-1,0,0
7,Корпорация монстров,2001,США,"мультфильм, фэнтези",8.1,367 188,https://www.kinopoisk.ru/film/458/,-1,0,0
8,Доктор Хаус,2012,США,"драма, детектив",8.8,365 635,https://www.kinopoisk.ru/film/178710/,-1,0,0


In [18]:
def df_to_str(df):
    df = df.set_index('name')
    s = ''
    for i in range(len(df)):
        name = df.index[i]
        s += '<b>' + str(name) + '</b>\\n\\n'
        s += 'Рейтинг' + ' ' * 17 + '-' + ' ' * 5 + str(df.loc[name]['rating']) + '\\n'
        s += 'Кол-во оценок     ' + '-' + ' ' * 5 + str(df.loc[name]['rating_count']) + '\\n'
        genre_list = str(df.loc[name]['genre']).split(', ')
        s += 'Жанр' + ' ' * 22 + '-' + ' ' * 5 + genre_list[0]
        if len(genre_list) == 2:
            s += '\\n' + ' ' * 39 + genre_list[1]
        s += '\\n'
        if i != len(df) - 1:
            s += '\\n\\n\\n'
    return(s)
s = df_to_str(d1)
#print(s)

In [35]:
d1

,name,date,country,genre,rating,rating_count,link,audio,subtitles,description,users
0,Ford против Ferrari,2019,США,"биография, спорт",8.2,398 824,https://www.kinopoisk.ru/film/835086/,0,0,0,0
1,Легенда,2015,"Великобритания, Франция","криминал, триллер",7.2,388 194,https://www.kinopoisk.ru/film/839954/,0,0,0,0
2,Алита: Боевой ангел,2019,"США, Япония","фантастика, боевик",7.1,385 245,https://www.kinopoisk.ru/film/88173/,0,0,0,0
3,Довод,2020,"Великобритания, США","фантастика, боевик",7.6,385 139,https://www.kinopoisk.ru/film/1236063/,0,0,0,0
4,Пираты Карибского моря: На краю света,2007,США,"приключения, боевик",8.0,379 904,https://www.kinopoisk.ru/film/102124/,0,0,0,0
5,Дэдпул 2,2018,"США, Канада","фантастика, боевик",7.4,376 663,https://www.kinopoisk.ru/film/961715/,0,0,0,0
6,Богемская рапсодия,2018,"Великобритания, США","биография, музыка",8.0,375 409,https://www.kinopoisk.ru/film/568289/,0,0,0,0
7,Корпорация монстров,2001,США,"мультфильм, фэнтези",8.1,367 446,https://www.kinopoisk.ru/film/458/,0,0,0,6
8,Доктор Хаус,2012,США,"драма, детектив",8.8,365 879,https://www.kinopoisk.ru/film/178710/,0,0,0,6
9,Великий Гэтсби,2013,"Австралия, США","драма, мелодрама",7.9,363 481,https://www.kinopoisk.ru/film/463724/,0,0,0,6


In [25]:
s = 'additional_info'
s1 = 'additional_infomy_films'
s1[:15] == s
print(s1[15:])

my_films


In [115]:
#df1.loc[(df1.name == 'Легенда'), ['audio', 'link']] = ['aa', 'dd']
#df1
print(df1)
name = 'Дэдпул 2'
if name in list(df1.name):
    index = df1.loc[(df1.name == name)].index[0]
    elem = df1.iloc[index]
    print(elem['name'])
    df1.iloc[index]['link'] = 'asfsafdasdfafsdasdfasfd'
    if type(df1) != str:
        print('asfda')
print(df1)

                                     name  date                  country  \
0                     Ford против Ferrari  2019                      США   
1                                 Легенда  2015  Великобритания, Франция   
2                                   Довод  2020      Великобритания, США   
3                     Алита: Боевой ангел  2019              США, Япония   
4   Пираты Карибского моря: На краю света  2007                      США   
5                                Дэдпул 2  2018              США, Канада   
6                      Богемская рапсодия  2018      Великобритания, США   
10                          Тор: Рагнарёк  2017           США, Австралия   
11                            Живая сталь  2011               США, Индия   
12                            Я – легенда  2007                      США   
13           Вечное сияние чистого разума  2004                      США   
14                             День сурка  1993                      США   
15          

In [113]:
df1.loc[1]

name                            Легенда
date                               2015
country         Великобритания, Франция
genre                 криминал, триллер
rating                              7.2
rating_count                    388 594
link                                asf
audio                                aa
subtitles                             0
description                           0
users                                 1
Name: 1, dtype: object

In [21]:
df1 = d1
df2 = d2
df1['users'] += df1['name'].isin(df2['name'])
df1 = pd.concat([df1, df2]).drop_duplicates('name').reset_index(drop=True)
df1 = df1[df1.users == 1]
print(df1)

                                     name  date                  country  \
0                     Ford против Ferrari  2019                      США   
1                                 Легенда  2015  Великобритания, Франция   
2                                   Довод  2020      Великобритания, США   
3                     Алита: Боевой ангел  2019              США, Япония   
4   Пираты Карибского моря: На краю света  2007                      США   
5                                Дэдпул 2  2018              США, Канада   
6                      Богемская рапсодия  2018      Великобритания, США   
10                          Тор: Рагнарёк  2017           США, Австралия   
11                            Живая сталь  2011               США, Индия   
12                            Я – легенда  2007                      США   
13           Вечное сияние чистого разума  2004                      США   
14                             День сурка  1993                      США   
15          

In [118]:
df_films = pd.read_csv("df_films.csv")
len(df_films)

60

In [9]:
df1 = d1
df2 = d2
#df1.to_csv('csv_test.csv', index=False)
df1 = pd.read_csv("csv_test.csv")
#s = df_to_str(df1)
#print(df1)
#df1 = df1.set_index('name')
#df1 = df1[df1['name'].isin(df2['name']) == True]
#df1 = df1.drop(index)
# print('\n\n')
print(type(df1.iloc[1]['audio']))
df1['audio'] = 'ss'
print(type(df1.iloc[1]['audio']))
print(df1)

<class 'numpy.int64'>
<class 'str'>
                  name  date                  country               genre  \
0              Легенда  2015  Великобритания, Франция   криминал, триллер   
1  Алита: Боевой ангел  2019              США, Япония  фантастика, боевик   
2                Довод  2020      Великобритания, США  фантастика, боевик   

   rating rating_count                                    link audio  \
0     7.2      387 977   https://www.kinopoisk.ru/film/839954/    ss   
1     7.1      384 969    https://www.kinopoisk.ru/film/88173/    ss   
2     7.6      384 587  https://www.kinopoisk.ru/film/1236063/    ss   

   subtitles  description  
0          0            0  
1          0            0  
2          0            0  
